<a href="https://colab.research.google.com/github/0thanh2000/44/blob/main/11a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# 0. MOUNT DRIVE VÀ IMPORT THƯ VIỆN
# -----------------------------
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import gc
import joblib
import matplotlib.pyplot as plt
import sys, logging

from datetime import timedelta
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Dropout, LayerNormalization, MultiHeadAttention, Add, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import RobustScaler

# -----------------------------
# 1. THIẾT LẬP THAM SỐ & ĐỌC DỮ LIỆU
# -----------------------------
# Các tham số chính:
window_m5   = 144    # Số nến M5
window_m30  = 120    # Số nến M30
window_h2   = 200    # Số nến H2
num_features = 4     # Số đặc trưng: Open, High, Low, Close
dtype = np.float32   # Kiểu dữ liệu sử dụng
batch_size = 32      # Kích thước batch

# Upload file CSV (nhớ upload file lên Colab trước khi chạy)
from google.colab import files
uploaded = files.upload()

# Đọc dữ liệu từ các file CSV
df_m5  = pd.read_csv('GBPUSD_M5a.csv', parse_dates=['Datetime'])
df_m30 = pd.read_csv('GBPUSD_M30a.csv', parse_dates=['Datetime'])
df_h2  = pd.read_csv('GBPUSD_H2a.csv', parse_dates=['Datetime'])

# Sắp xếp theo thời gian, reset index và loại bỏ giá trị thiếu
for df in [df_m5, df_m30, df_h2]:
    df.sort_values('Datetime', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.dropna(inplace=True)

In [ ]:
# 2. XỬ LÝ DỮ LIỆU: TẠO TARGET INDICES VÀ CHIA TRAIN-TEST
# -----------------------------
# Hàm lấy lịch sử dữ liệu của từng khung thời gian (dùng cho cả training và backtest)
def get_history(target_time, df, window):
    mask = df['Datetime'] < target_time
    if mask.sum() < window:
        return None
    return df.loc[mask].tail(window)[['Open', 'High', 'Low', 'Close']].values

# Tạo danh sách các index hợp lệ từ dữ liệu H2
valid_indices = []
for i in range(window_h2, len(df_h2) - 1):
    target_time = df_h2.iloc[i+1]['Datetime']
    if (get_history(target_time, df_m5, window_m5) is not None and
        get_history(target_time, df_m30, window_m30) is not None):
        valid_indices.append(i)

# Chia train-test theo thời gian (25% đầu làm test, phần còn lại làm train)
split_idx = int(len(valid_indices) * 0.25)
test_indices = valid_indices[:split_idx]
train_indices = valid_indices[split_idx:]


In [ ]:
# 3. KHỞI TẠO SCALER SỬ DỤNG TẬP TRAIN
# -----------------------------
def init_scalers(indices):
    m5_data, m30_data, h2_data, y_data = [], [], [], []
    for idx in indices:
        target_time = df_h2.iloc[idx+1]['Datetime']
        m5   = get_history(target_time, df_m5, window_m5)
        m30  = get_history(target_time, df_m30, window_m30)
        h2   = df_h2.iloc[idx-window_h2+1:idx+1][['Open', 'High', 'Low', 'Close']].values
        y    = df_h2.iloc[idx+1]['Close']

        m5_data.append(m5)
        m30_data.append(m30)
        h2_data.append(h2)
        y_data.append(y)

    scaler_m5 = RobustScaler()
    scaler_m5.fit(np.vstack(m5_data).reshape(-1, num_features))

    scaler_m30 = RobustScaler()
    scaler_m30.fit(np.vstack(m30_data).reshape(-1, num_features))

    scaler_h2 = RobustScaler()
    scaler_h2.fit(np.vstack(h2_data).reshape(-1, num_features))

    scaler_y = RobustScaler()
    scaler_y.fit(np.array(y_data).reshape(-1, 1))

    return scaler_m5, scaler_m30, scaler_h2, scaler_y

scaler_m5, scaler_m30, scaler_h2, scaler_y = init_scalers(train_indices)
gc.collect()

In [ ]:
# 4. XÂY DỰNG tf.data.Dataset TỪ GENERATOR (KHÔNG SHUFFLE)
# -----------------------------
def sample_generator(indices):
    """
    Generator trả về từng sample với input là dict chứa dữ liệu của các khung (M5, M30, H2)
    và target là giá (đã được chuẩn hóa).
    """
    for idx in indices:
        target_time = df_h2.iloc[idx+1]['Datetime']
        m5 = get_history(target_time, df_m5, window_m5)
        m30 = get_history(target_time, df_m30, window_m30)
        h2 = df_h2.iloc[idx-window_h2+1:idx+1][['Open', 'High', 'Low', 'Close']].values
        y  = df_h2.iloc[idx+1]['Close']

        # Chuẩn hóa và chuyển sang kiểu dữ liệu đã định nghĩa
        m5_scaled   = scaler_m5.transform(m5.reshape(-1, num_features)).reshape(m5.shape).astype(dtype)
        m30_scaled  = scaler_m30.transform(m30.reshape(-1, num_features)).reshape(m30.shape).astype(dtype)
        h2_scaled   = scaler_h2.transform(h2.reshape(-1, num_features)).reshape(h2.shape).astype(dtype)
        y_scaled    = scaler_y.transform([[y]]).flatten().astype(dtype)

        yield (
            {
                'input_m5': m5_scaled,
                'input_m30': m30_scaled,
                'input_h2': h2_scaled
            },
            y_scaled
        )

# Định nghĩa cấu trúc đầu ra cho tf.data.Dataset
output_signature = (
    {
        'input_m5': tf.TensorSpec(shape=(window_m5, num_features), dtype=tf.float32),
        'input_m30': tf.TensorSpec(shape=(window_m30, num_features), dtype=tf.float32),
        'input_h2': tf.TensorSpec(shape=(window_h2, num_features), dtype=tf.float32)
    },
    tf.TensorSpec(shape=(), dtype=tf.float32)
)

# Tạo Dataset cho tập train và test (không dùng shuffle, giữ nguyên thứ tự)
train_dataset = tf.data.Dataset.from_generator(lambda: sample_generator(train_indices),
                                                 output_signature=output_signature)
test_dataset = tf.data.Dataset.from_generator(lambda: sample_generator(test_indices),
                                                output_signature=output_signature)

# Áp dụng batch và prefetch để tối ưu quá trình huấn luyện
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
# 5. XÂY DỰNG MÔ HÌNH (Multi-branch với LSTM + MultiHeadAttention)
# -----------------------------
def build_branch(input_shape, name_prefix):
    input_layer = Input(shape=input_shape, name=f'input_{name_prefix}', dtype=dtype)
    x = LSTM(64, return_sequences=True, name=f'lstm_{name_prefix}')(input_layer)
    x_attn = MultiHeadAttention(num_heads=4, key_dim=16, name=f'mha_{name_prefix}')(x, x)
    x = Add(name=f'add_{name_prefix}')([x, x_attn])
    x = LayerNormalization(name=f'ln_{name_prefix}')(x)
    # Global Average Pooling theo chiều thời gian
    x = Lambda(lambda t: tf.reduce_mean(t, axis=1), name=f'global_avg_{name_prefix}')(x)
    x = Dropout(0.01, name=f'dropout_{name_prefix}')(x)
    return input_layer, x

# Xây dựng các nhánh cho M5, M30 và H2
input_m5_layer, branch_m5 = build_branch((window_m5, num_features), 'm5')
input_m30_layer, branch_m30 = build_branch((window_m30, num_features), 'm30')
input_h2_layer, branch_h2 = build_branch((window_h2, num_features), 'h2')

# Fusion Attention: hợp nhất các đặc trưng từ 3 nhánh
combined = Concatenate(name='concatenate')([branch_m5, branch_m30, branch_h2])
fusion_query = Lambda(lambda t: tf.expand_dims(t, axis=1), name='fusion_expand_dims')(combined)
fusion_attn = MultiHeadAttention(num_heads=4, key_dim=16, name='fusion_mha')(fusion_query, fusion_query)
fusion_attn = Lambda(lambda t: tf.squeeze(t, axis=1), name='fusion_squeeze')(fusion_attn)
fusion_out = Add(name='fusion_add')([combined, fusion_attn])
fusion_out = LayerNormalization(name='fusion_ln')(fusion_out)
fusion_out = Dropout(0.01, name='fusion_dropout')(fusion_out)

# Các lớp Dense cuối cùng
x = Dense(64, activation='relu', name='dense_1')(fusion_out)
x = LayerNormalization(name='dense_ln')(x)
x = Dropout(0.01, name='dense_dropout')(x)
output = Dense(1, activation='linear', name='output')(x)

# Xây dựng mô hình
model = Model(inputs=[input_m5_layer, input_m30_layer, input_h2_layer], outputs=output)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# Sử dụng Huber Loss – ổn định với dữ liệu nhiễu
model.compile(optimizer=optimizer,
              loss='mae',
              metrics=['mse'])

model.summary()

In [ ]:
# 6. HUẤN LUYỆN MÔ HÌNH VỚI tf.data.Dataset
# -----------------------------
# Cài đặt callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=8, min_delta=0, restore_best_weights=True)
checkpoint = ModelCheckpoint('/content/drive/MyDrive/7aEUa.keras', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-8, verbose=1)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100,
    callbacks=[early_stop, checkpoint, reduce_lr]
)


In [ ]:
# 7. LƯU SCALERS VÀ MODEL
# -----------------------------
joblib.dump(scaler_m5, '/content/drive/MyDrive/scaler_m5.joblib')
joblib.dump(scaler_m30, '/content/drive/MyDrive/scaler_m30.joblib')
joblib.dump(scaler_h2, '/content/drive/MyDrive/scaler_h2.joblib')
joblib.dump(scaler_y, '/content/drive/MyDrive/scaler_y.joblib')
model.save('/content/drive/MyDrive/full_model.keras')

# Giải phóng bộ nhớ
del df_m5, df_m30, df_h2
gc.collect()

In [ ]:
# 8. ĐÁNH GIÁ VÀ TRỰC QUAN HÓA KẾT QUẢ HUẤN LUYỆN
# -----------------------------
# Lấy dự báo từ test_dataset
y_pred = model.predict(test_dataset)
y_pred = y_pred.flatten()

# Vì dữ liệu target trong generator đã được chuẩn hóa, thực hiện inverse transform:
y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()

# Tạo mảng ground truth (y_test) từ test_indices
# Lấy giá từ df_h2 (lưu ý: nếu bạn đã xóa df_h2, bạn cần lưu lại giá trị gốc trước)
# Ở đây, ta tái tạo y_test từ test_indices (giả sử bạn vẫn có file gốc)
df_h2_raw = pd.read_csv('GBPUSD_H2a.csv', parse_dates=['Datetime'])
df_h2_raw.sort_values('Datetime', inplace=True)
df_h2_raw.reset_index(drop=True, inplace=True)
df_h2_raw.dropna(inplace=True)

y_test_list = []
for idx in test_indices:
    y_val = df_h2_raw.iloc[idx+1]['Close']
    y_test_list.append(y_val)
y_test_array = np.array(y_test_list)

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, median_absolute_error
mae   = mean_absolute_error(y_test_array, y_pred)
mse   = mean_squared_error(y_test_array, y_pred)
mape  = mean_absolute_percentage_error(y_test_array, y_pred)
r2    = r2_score(y_test_array, y_pred)
medae = median_absolute_error(y_test_array, y_pred)

# Tính Direction Accuracy: tỷ lệ dự báo đúng hướng thay đổi của giá
actual_diff = np.diff(y_test_array)
pred_diff   = np.diff(y_pred)
direction_accuracy = np.mean(np.sign(actual_diff) == np.sign(pred_diff))

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("R² Score:", r2)
print("Median Absolute Error:", medae)
print("Direction Accuracy:", direction_accuracy)

# Vẽ biểu đồ Loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# Vẽ biểu đồ so sánh giá thực tế và giá dự báo
plt.figure(figsize=(12, 6))
plt.plot(y_test_array, label="Giá thực tế", color="blue", linestyle="-", marker="o")
plt.plot(y_pred, label="Giá dự báo", color="red", linestyle="--", marker="x")
plt.xlabel("Chỉ số mẫu")
plt.ylabel("Giá")
plt.title("So sánh Giá thực tế và Giá dự báo")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 9. PHẦN BACKTEST (TUỲ CHỌN)
# -----------------------------
# Cấu hình logging cho backtest
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def read_data(csv_path):
    try:
        logging.info("Đọc dữ liệu từ file CSV: %s", csv_path)
        df = pd.read_csv(csv_path, parse_dates=["Datetime"])
    except Exception as e:
        logging.error("Không thể đọc file CSV: %s", e)
        sys.exit(1)
    df.sort_values("Datetime", inplace=True)
    required_cols = ["Datetime", "Open", "High", "Low", "Close"]
    for col in required_cols:
        if col not in df.columns:
            logging.error("File CSV thiếu cột bắt buộc: %s", col)
            sys.exit(1)
    if "Volume" in df.columns:
        logging.info("Loại bỏ cột Volume khỏi dữ liệu.")
    df = df[required_cols]
    return df

def get_history_backtest(target_time, df, window):
    df_hist = df[df['Datetime'] < target_time].tail(window)
    if len(df_hist) == window:
        return df_hist[['Open', 'High', 'Low', 'Close']].values
    else:
        return None

def create_sequences_from_df(df, window_m5, window_m30, window_h2, horizon):
    X_m5, X_m30, X_h2, Y, time_list = [], [], [], [], []
    end_index = len(df) - horizon + 1
    for i in range(window_h2, end_index):
        target_seq = df.loc[i : i+horizon-1, "Close"].values
        target_time = df.loc[i+horizon-1, "Datetime"]
        seq_m5 = get_history_backtest(target_time, df, window_m5)
        seq_m30 = get_history_backtest(target_time, df, window_m30)
        seq_h2 = df.loc[i-window_h2+1 : i, ["Open", "High", "Low", "Close"]].values
        if (seq_m5 is not None) and (seq_m30 is not None) and (len(seq_h2)==window_h2):
            X_m5.append(seq_m5)
            X_m30.append(seq_m30)
            X_h2.append(seq_h2)
            Y.append(target_seq)
            time_list.append(target_time)
    return np.array(X_m5), np.array(X_m30), np.array(X_h2), np.array(Y), np.array(time_list)

def transform_3d_array(X, scaler):
    shape = X.shape
    X_reshaped = X.reshape(-1, shape[-1])
    X_scaled = scaler.transform(X_reshaped).reshape(shape)
    return X_scaled

def compute_directional_accuracy(y_true, y_pred):
    diff_true = np.diff(y_true[:, -1], axis=0)
    diff_pred = np.diff(y_pred[:, -1], axis=0)
    correct = np.sum(np.sign(diff_true) == np.sign(diff_pred))
    return correct / len(diff_true)

def save_predictions_to_csv(dates, actual, predicted, output_path):
    n, horizon = actual.shape
    columns = ["Datetime"] + [f"Actual_{i+1}" for i in range(horizon)] + [f"Predicted_{i+1}" for i in range(horizon)]
    data = []
    for i in range(n):
        row = [dates[i]] + list(actual[i]) + list(predicted[i])
        data.append(row)
    df_out = pd.DataFrame(data, columns=columns)
    try:
        df_out.to_csv(output_path, index=False)
        logging.info("Đã lưu kết quả dự báo vào file: %s", output_path)
    except Exception as e:
        logging.error("Lỗi khi lưu kết quả dự báo: %s", e)

def backtest():
    # Cấu hình đường dẫn cho file backtest và các scaler, model đã lưu
    csv_path = "/content/drive/MyDrive/UJ_M15u_backtest.csv"
    scaler_m5_path = "/content/drive/MyDrive/scaler_m5.save"
    scaler_m30_path = "/content/drive/MyDrive/scaler_m30.save"
    scaler_h2_path = "/content/drive/MyDrive/scaler_h2.save"
    scaler_y_path = "/content/drive/MyDrive/scaler_y.save"
    model_path = "/content/drive/MyDrive/UJ_finam288_1.keras"
    output_csv = "/content/drive/MyDrive/backtest_results.csv"

    window_m5 = 144
    window_m30 = 120
    window_h2 = 200
    horizon = 8

    df = read_data(csv_path)
    try:
        logging.info("Trích xuất features và target từ dữ liệu backtest.")
    except Exception as e:
        logging.error("Lỗi khi trích xuất features: %s", e)
        sys.exit(1)

    try:
        from joblib import load
        logging.info("Loading scaler M5 từ: %s", scaler_m5_path)
        scaler_m5 = load(scaler_m5_path)
        logging.info("Loading scaler M30 từ: %s", scaler_m30_path)
        scaler_m30 = load(scaler_m30_path)
        logging.info("Loading scaler H2 từ: %s", scaler_h2_path)
        scaler_h2 = load(scaler_h2_path)
        logging.info("Loading scaler y từ: %s", scaler_y_path)
        scaler_y = load(scaler_y_path)
    except Exception as e:
        logging.error("Lỗi khi load scaler: %s", e)
        sys.exit(1)

    X_m5, X_m30, X_h2, y_seq, time_seq = create_sequences_from_df(df, window_m5, window_m30, window_h2, horizon)
    logging.info("X_m5.shape = %s", X_m5.shape)
    logging.info("X_m30.shape = %s", X_m30.shape)
    logging.info("X_h2.shape = %s", X_h2.shape)
    logging.info("y_seq.shape = %s", y_seq.shape)

    X_m5_scaled = transform_3d_array(X_m5, scaler_m5)
    X_m30_scaled = transform_3d_array(X_m30, scaler_m30)
    X_h2_scaled = transform_3d_array(X_h2, scaler_h2)
    y_seq_scaled = scaler_y.transform(y_seq.reshape(-1, 1)).reshape(y_seq.shape)

    try:
        logging.info("Loading mô hình từ: %s", model_path)
        model = tf.keras.models.load_model(model_path)
    except Exception as e:
        logging.error("Lỗi khi load mô hình: %s", e)
        sys.exit(1)

    try:
        predictions_scaled = model.predict([X_m5_scaled, X_m30_scaled, X_h2_scaled])
        logging.info("predictions_scaled.shape = %s", predictions_scaled.shape)
    except Exception as e:
        logging.error("Lỗi khi chạy dự đoán: %s", e)
        sys.exit(1)

    try:
        y_inversed = scaler_y.inverse_transform(y_seq_scaled.reshape(-1, 1)).reshape(y_seq_scaled.shape)
        pred_inversed = scaler_y.inverse_transform(predictions_scaled.reshape(-1, 1)).reshape(predictions_scaled.shape)
    except Exception as e:
        logging.error("Lỗi khi inverse transform: %s", e)
        sys.exit(1)

    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score, median_absolute_error
    mae = mean_absolute_error(y_inversed, pred_inversed)
    rmse = np.sqrt(mean_squared_error(y_inversed, pred_inversed))
    mape_value = mean_absolute_percentage_error(y_inversed, pred_inversed) * 100
    r2 = r2_score(y_inversed, pred_inversed)
    medae = median_absolute_error(y_inversed, pred_inversed)
    dir_acc = compute_directional_accuracy(y_inversed, pred_inversed)

    logging.info("MAE (horizon %d): %.4f", horizon, mae)
    logging.info("RMSE (horizon %d): %.4f", horizon, rmse)
    logging.info("MAPE: %.2f%%", mape_value)
    logging.info("R²: %.4f", r2)
    logging.info("Median Absolute Error: %.4f", medae)
    logging.info("Directional Accuracy (bước cuối horizon): %.2f%%", dir_acc * 100)

    print("MAE (horizon {}): {:.4f}".format(horizon, mae))
    print("RMSE (horizon {}): {:.4f}".format(horizon, rmse))
    print("MAPE: {:.2f}%".format(mape_value))
    print("R²: {:.4f}".format(r2))
    print("Median Absolute Error: {:.4f}".format(medae))
    print("Directional Accuracy (bước cuối horizon): {:.2f}%".format(dir_acc * 100))

    if time_seq is not None:
        save_predictions_to_csv(time_seq, y_inversed, pred_inversed, output_csv)
    else:
        n_samples = y_inversed.shape[0]
        df_pred = pd.DataFrame({"Index": np.arange(n_samples)})
        for i in range(horizon):
            df_pred[f"Actual_{i+1}"] = y_inversed[:, i]
            df_pred[f"Predicted_{i+1}"] = pred_inversed[:, i]
        try:
            df_pred.to_csv(output_csv, index=False)
            logging.info("Đã lưu kết quả dự báo vào file: %s", output_csv)
        except Exception as e:
            logging.error("Lỗi khi lưu kết quả dự báo: %s", e)

    plt.figure(figsize=(14,7))
    if time_seq is not None:
        plt.plot(time_seq, y_inversed[:, -1], label="Giá thực tế (bước cuối)", color="blue")
        plt.plot(time_seq, pred_inversed[:, -1], label="Dự báo (bước cuối)", linestyle="--", color="red")
        plt.xlabel("Datetime")
    else:
        plt.plot(y_inversed[:, -1], label="Giá thực tế (bước cuối)", color="blue")
        plt.plot(pred_inversed[:, -1], label="Dự báo (bước cuối)", linestyle="--", color="red")
        plt.xlabel("Index")
    plt.ylabel("Giá")
    plt.title("So sánh Giá thực tế và Giá dự báo (bước cuối của horizon)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Để chạy phần backtest, chỉ cần gọi:
# backtest()